#### This assignment may be worked individually or in pairs. Enter your name/s here:
    

In [29]:
#alexa y antonio

# Assignment 2: Decision Trees

In this assignment we'll implement the Decision Tree algorithm to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the Diabetic Retinopathy data set, which contains features from the Messidor image set to predict whether an image contains signs of diabetic retinopathy or not. This dataset has `1150` instances and `20` attributes (some categorical, some continuous). You can find additional details about the dataset [here](http://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set).

Attribute Information:

    0) The binary result of quality assessment. 0=bad quality 1=sufficient quality.

    1) The binary result of pre-screening, where 1 indicates severe retinal abnormality and 0 its lack. 

    2-7) The results of MA detection. Each feature value stand for the number of MAs found at the confidence levels alpha = 0.5, . . . , 1, respectively. 

    8-15) Contains the same information as 2-7, but for exudates. However, as exudates are represented by a set of points rather than the number of pixels constructing the lesions, these features are normalized by dividing the number of lesions with the diameter of the region of interest (ROI) to compensate for different image sizes. 

    16) The euclidean distance between the center of the macula and the center of the optic disc. This feature is also normalized with the diameter of the ROI.

    17) The diameter of the optic disc. 

    18) Result of the AM/FM-based (amplitude-modulation frequency-modulation) imaging. 0=normal and 1=abnormal.

    19) Class label. 1 = contains signs of Diabetic Retinopathy, 0 = no signs of Diabetic Retinopathy.

#### Implementation: 
The function prototypes are given to you, please don't change these. You can add additional helper functions if needed. 

*Suggestion:* The dataset is substantially big, for the purpose of easy debugging, work with a subset of the data and test your decision tree implementation on that.

#### Notes:
Parts of this assignment will be **autograded** so a couple of caveats :-
- Entropy is calculated using log with base 2, `math.log2(x)`.
- For continuous features ensure that the threshold value lies exactly between 2 values. For example, if for feature 2 the best split occurs between 10 and 15 then the threshold value will be set as 12.5. For binary features [0/1] the threshold value will be 0.5.
- All values < `thresh_val` go to the left child and all values >= `thresh_val` go to the right child.

In [1]:
# Standard Headers
# You are welcome to add additional headers if you wish
# EXCEPT for scikit-learn... You may NOT use scikit-learn for this assignment!
import pandas as pd
from math import log2
import time

In [3]:
class TreeNode:
    is_leaf = True          # boolean variable to check if the node is a leaf
    feature_idx = None      # index that identifies the feature
    thresh_val = None       # threshold value that splits the node
    prediction = None       # prediction class (only valid for leaf nodes)
    left_child = None       # left TreeNode (all values < thresh_val)
    right_child = None      # right TreeNode (all values >= thresh_val)
    
    def printTree(self, level=0):    # for debugging purposes
        if self.is_leaf:
            print ('-'*level + 'Leaf Node:      predicts ' + str(self.prediction))
        else:
            print ('-'*level + 'Internal Node:  splits on feature ' 
                   + str(self.feature_idx) + ' with threshold ' + str(self.thresh_val))
            self.left_child.printTree(level+1)
            self.right_child.printTree(level+1)

Q1. Implement the function `make_prediction` that takes the decision tree root and a data point instance and returns the prediction label.

In [44]:
def make_prediction(tree_root, data_point):
    if tree_root.is_leaf:
        return tree_root.prediction
    else:
        if data_point < tree_root.thresh_val:
            make_prediction(tree_root.left_child, data_point)
        else:
            make_prediction(tree_root.right_child, data_point)
    return None

Q2. Implement the function `split_dataset` given an input data set, a `feature_idx` and the `threshold` for the feature. `left_split` will have all values < `threshold` and `right_split` will have all values >= `threshold`.

In [33]:
def split_dataset(data, feature_idx, threshold):
    left_split = []
    right_split = []
    #your code goes here
    for i in data.iloc[:, feature_idx]:
        if i < threshold:
            left_split.append(i)
        else:
            right_split.append(i)
            
    return (left_split, right_split)

Q3. Implement the function `calc_entropy` to return the entropy of the input dataset.

In [34]:
def calc_entropy(data):
    entropy = 0.0
    #your code goes here
    left, right = split_dataset(data, -1, 0.5)

    def sub_entropy(sub):
        return - len(sub) / len(data.iloc[:, -1]) * log2(len(sub) / len(data.iloc[:, -1]))
        
    entropy = sub_entropy(left) + sub_entropy(right)
    return entropy

Q4. Implement the function `calc_best_threshold` which returns the best information gain and the corresponding threshold value for one feature at `feature_idx`.

In [ ]:
def calc_best_threshold(data, feature_idx):
    best_info_gain = 0.0
    best_thresh = None
    #your code goes here
    data = data.sort_values(by=feature_idx)
    previous = None

    def info_gain(threshold):
        # entropy parent - sum of entropy child nodes
        left, right = split_dataset(data, feature_idx, threshold)
        return calc_entropy(data) - calc_entropy(left) - calc_entropy(right)
    
    for i, value in enumerate(data.iloc[:, -1]):
        if value == previous: # no threshold
            continue
        else: # threshold
            threshold = (data.iloc[i, feature_idx] + data.iloc[i-1, feature_idx]) / 2

            if (threshold == data.iloc[i-1, feature_idx]) or (threshold == data.iloc[i+1, feature_idx]):
                continue
            
            if info_gain(threshold) >= best_info_gain:
                best_info_gain = info_gain(threshold)
                best_thresh = threshold
            
        if i != 0:
            previous = value

    return (best_info_gain, best_thresh)

Q5. Implement the function `identify_best_split` which returns the best feature to split on for an input dataset, and also returns the corresponding threshold value.

In [36]:
def identify_best_split(data):
    if len(data) < 2:
        return (None, None)
    best_feature = None
    best_thresh = None
    #your code goes here
    for feature_idx in len(data.columns):
        if feature_idx == 1:
            best_feature = 1
            best_info_gain, best_thresh = calc_best_threshold(data,1)
        else:
            info_gain, thresh = calc_best_threshold(data,feature_idx)
            if info_gain > best_info_gain:
                best_info_gain =  info_gain
                best_thresh = thresh
   
    return (best_feature, best_thresh)

Q6. Implement the function `create_leaf_node` which returns a `TreeNode` with `is_leaf=True` and `prediction` set to whichever classification occurs most in the dataset at this node. If there is a tie, choose classification label 1 (has disease). 

In [37]:
def create_leaf_node(data):        
    #your code goes here
    # is_leaf = True          # boolean variable to check if the node is a leaf
    # feature_idx = None      # index that identifies the feature
    # thresh_val = None       # threshold value that splits the node
    # prediction = None       # prediction class (only valid for leaf nodes)
    # left_child = None       # left TreeNode (all values < thresh_val)
    # right_child = None      # right TreeNode (all values >= thresh_val)

    leaf_node = TreeNode()
    leaf_node.is_leaf = True
    no_disease = len(data[data[-1] == 0])    
    disease = len(data[data[-1] == 1])  
    if no_disease > disease:
        leaf_node.prediction = 0
    elif disease > no_disease:
        leaf_node.prediction = 1
    else:
        leaf_node.prediction = 1
    return None

Q7. Implement the `create_decision_tree` function. `max_levels` denotes the maximum height of the tree - For example, if `max_levels = 1` then the decision tree will only contain the leaf node at the root. [Hint: this is where the recursion happens.]

In [38]:
def create_decision_tree(data, max_levels):
    #your code goes here
    return None

Q8. Given a decision tree and a test set, the function `calc_accuracy` returns the accuracy of the classifier. You'll use the `make_prediction` function for this.

In [39]:
def calc_accuracy(tree_root, test_data):
    #your code goes here
    return 0.0

Q9. Now measure the accuracy of using a decision tree on this data with a 5-fold cross validation. 
Set the `max_levels` parameter to 10 and print the accuracy from a 5-fold-CV.

This must run in under 10 minutes, otherwise points will be deducted. 

In [ ]:
def run_CV(filename):
    start = time.time()
    
    # read in data
    d = pd.read_csv(filename, header = None)

    #your code goes here

    end = time.time()
    print ('Time taken:', end - start)


run_CV("messidor_features.txt")